# Logistic Regression

In [1]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
ORIGINAL_DATA_DIR = "C:/NLP/Assignment4/handout/data"
print(ORIGINAL_DATA_DIR)
BERT_FEATURE_DIR = "C:/NLP/Assignment4/handout/bert_output_data"

C:/NLP/Assignment4/handout/data


## Format training data

`X` will be a matrix with `N` rows for the `N` texts in the training data, and `M` columns for the `M` features generated by BERT.

`y` will be an array of `N` class labels for training.

In [3]:
train_df = pd.read_csv(ORIGINAL_DATA_DIR+"/lang_id_train.csv")
train_df.shape

(6000, 2)

In [4]:
test_df = pd.read_csv(ORIGINAL_DATA_DIR+"/lang_id_test.csv")
test_df.shape

(2000, 2)

In [5]:
bert_vectors = []
with open(os.path.join(BERT_FEATURE_DIR, "train.jsonlines"), "rt") as infile:
    for line in infile:
        bert_data = json.loads(line)
        for t in bert_data["features"]:
            # Only extract the [CLS] vector used for classification
            if t["token"] == "[CLS]":
                # We only use the representation at the final layer of the network
                bert_vectors.append(t["layers"][0]["values"])
                break

In [6]:
bert_vectors_test = []
with open(os.path.join(BERT_FEATURE_DIR, "test.jsonlines"), "rt") as infile:
    for line in infile:
        bert_data = json.loads(line)
        for t in bert_data["features"]:
            # Only extract the [CLS] vector used for classification
            if t["token"] == "[CLS]":
                # We only use the representation at the final layer of the network
                bert_vectors_test.append(t["layers"][0]["values"])
                break

In [7]:
len(bert_vectors)

6000

In [8]:
len(bert_vectors_test)

2000

In [9]:
X_train = np.array(bert_vectors)
y_train = train_df["native_language"].values
X_test = np.array(bert_vectors_test)
y_test = test_df["native_language"].values

# Train logistic regression model

In [10]:
#lr_model = LogisticRegression(penalty="l2", C=1.0)
lr_model = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

lr_model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Test Logistic Regression Model

In [11]:
print (lr_model.intercept_, lr_model.coef_)

[ 0.01556567 -0.05973345 -0.06967004 -0.00044624  0.06709756 -0.09168432
  0.01396779  0.11227561 -0.06283247  0.07545989] [[-0.13366956  0.39882331  0.11291944 ... -0.24314519  0.20492511
   0.61365363]
 [-0.46007311 -0.04856964 -0.05859457 ... -0.14061645 -0.00399913
  -0.47984901]
 [ 0.35134614 -0.36917708 -0.10856648 ...  0.26340266 -0.18938361
  -0.18184642]
 ...
 [-0.34413203  0.07397263  0.0170779  ... -0.04083112  0.55463192
   0.25435844]
 [ 0.11415463 -0.11661916  0.26020379 ... -0.02403476 -0.17154475
   0.07271964]
 [ 0.05855351  0.34197147 -0.06833372 ...  0.551914   -0.06102568
   0.3211048 ]]


In [12]:
pred = lr_model.predict(X_test)
print(pred)

['Mandarin' 'Arabic' 'Mandarin' ... 'Korean' 'Arabic' 'Spanish']


In [14]:
Prob_value=lr_model.predict_proba(X_test)
print("Predicted Value")
print(Prob_value)

Predicted Value
[[1.58818181e-01 2.22267798e-01 1.63830662e-01 ... 8.01894510e-02
  3.28700979e-02 3.67867490e-02]
 [5.06554824e-01 8.38005974e-02 3.23352467e-02 ... 2.77086336e-02
  2.75934674e-02 1.03136888e-01]
 [1.44538315e-02 1.82150211e-01 3.93920859e-02 ... 4.22845634e-02
  5.18522641e-02 1.35692378e-01]
 ...
 [1.39229905e-01 2.10166117e-02 8.98570865e-02 ... 9.67388382e-02
  1.24187635e-01 1.37639416e-01]
 [7.68490844e-01 1.78718054e-02 1.29022054e-04 ... 4.97035645e-02
  1.53304002e-01 6.04751532e-03]
 [2.23888484e-01 1.88436804e-02 3.35418894e-03 ... 4.23673565e-01
  1.68430544e-02 2.49162461e-01]]


In [15]:
print("Accuracy : ",lr_model.score(X_test,y_test))

Accuracy :  0.459


In [21]:
results = confusion_matrix(y_test,pred)
print('Confusion Matrix :')
print(results)

Confusion Matrix :
[[ 99   6   6  11  14  14  11  25   5   9]
 [  9  68  16  16  37  12   5   7  15  15]
 [ 17  10  96  21  14  11   9   6   8   8]
 [  6  23  25  82  13   9  10   5  13  14]
 [ 12  49  16  16  63   3  15   8   5  13]
 [  9  13   5   6   3  99  32  16   7  10]
 [  7   7  13  10  10  21 110  13   0   9]
 [ 18   7  11   5  12  16  15 103   3  10]
 [ 14  12  10  10   8   3   0  12 114  17]
 [ 16  18  12  10  13  16  10  11  10  84]]


In [22]:
print('Accuracy Score :',accuracy_score(y_test,pred)) 

Accuracy Score : 0.459


In [23]:
print('Report : ')
print(classification_report(y_test,pred))

Report : 
              precision    recall  f1-score   support

      Arabic       0.48      0.49      0.49       200
   Cantonese       0.32      0.34      0.33       200
    Japanese       0.46      0.48      0.47       200
      Korean       0.44      0.41      0.42       200
    Mandarin       0.34      0.32      0.33       200
      Polish       0.49      0.49      0.49       200
     Russian       0.51      0.55      0.53       200
     Spanish       0.50      0.52      0.51       200
        Thai       0.63      0.57      0.60       200
  Vietnamese       0.44      0.42      0.43       200

    accuracy                           0.46      2000
   macro avg       0.46      0.46      0.46      2000
weighted avg       0.46      0.46      0.46      2000



In [24]:
print("F1-Score -Macro     - ", f1_score(y_test, pred, average='macro'))  
print("F1 Score - Micro    - ", f1_score(y_test, pred, average='micro'))
print("F1 Score - Weighted - ", f1_score(y_test, pred, average='weighted')) 

F1-Score -Macro     -  0.459037371220077
F1 Score - Micro    -  0.459
F1 Score - Weighted -  0.45903737122007704


In [25]:
print("precision score - Macro    - ",precision_score(y_test, pred, average='macro'))
print("precision score - Micro    - ",precision_score(y_test, pred, average='micro'))
print("precision score - Weighted - ",precision_score(y_test, pred, average='weighted'))

precision score - Macro    -  0.4600037960339021
precision score - Micro    -  0.459
precision score - Weighted -  0.4600037960339021


In [26]:
print("Recall Score - Macro    - ",recall_score(y_test, pred, average='macro'))
print("Recall Score - Micro    - ",recall_score(y_test, pred, average='micro'))
print("Recall Score - Weighted - ",recall_score(y_test, pred, average='weighted'))

Recall Score - Macro    -  0.45899999999999996
Recall Score - Micro    -  0.459
Recall Score - Weighted -  0.459
